In [151]:
import numpy as np 
import matplotlib.pyplot as plt  
import ipywidgets as widgets
from ipywidgets import GridspecLayout


In [180]:
lambd_init = 1.4 # arrival rate in orders/week 
mu_init = 1.5 # production rate in orders/week

cv_arrival_init = 1 # coefficient of variation of arrival times
bw_init = 1.2 # bullwhip effect (multiplication of variation per stage)

style = {'description_width': 'initial'}


lambd_semicon = widgets.FloatSlider(
    value=lambd_init,
    min=0.01,
    max=mu_init,
    step=0.1,
    description='Order arrival rate in k wafers/week',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    style=style)

mu_semicon = widgets.FloatSlider(
    value=mu_init,
    min=0.01,
    max=10,
    step=0.1,
    description='Production rate in k wafers/week',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    style=style)


cv_arrival_semicon = widgets.FloatSlider(
    value=cv_arrival_init*bw_init*bw_init,
    min=0.01,
    max=5,
    step=0.1,
    description='Coefficient of variation for order arrivals',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    style=style)


bullwhip_effect = widgets.FloatSlider(
    value=bw_init,
    min=1,
    max=5,
    step=0.1,
    description='Bullwhip effect (multiplication of variation per stage)',
    disabled=False,
    layout=widgets.Layout(width='500px', height="50px"),
    style=style)



lambd_oem = widgets.FloatSlider(
    value=lambd_init,
    min=0.01,
    max=mu_init,
    step=0.1,
    description='Order arrival rate in k cars/week',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    style=style)

mu_oem = widgets.FloatSlider(
    value=mu_init,
    min=0.01,
    max=10,
    step=0.1,
    description='Production rate in k cars/week',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    style=style)


cv_arrival_oem = widgets.FloatSlider(
    value=cv_arrival_init,
    min=0.01,
    max=5,
    step=0.1,
    description='Coefficient of variation for order arrivals',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    style=style)


lambd_tier1 = widgets.FloatSlider(
    value=lambd_init,
    min=0.01,
    max=mu_init,
    step=0.1,
    description='Order arrival rate in k parts/week',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    style=style)

mu_tier1 = widgets.FloatSlider(
    value=mu_init,
    min=0.01,
    max=10,
    step=0.1,
    description='Production rate in k parts/week',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    style=style)


cv_arrival_tier1 = widgets.FloatSlider(
    value=cv_arrival_init*bw_init,
    min=0.01,
    max=5,
    step=0.1,
    description='Coefficient of variation for order arrivals',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    style=style)


In [181]:
def drawPlot(p,W,p_chart,W_chart,W_chart_novariation,cv_arrival):
    plt.plot(p_chart*100,W_chart) 
    plt.plot(p_chart*100,W_chart_novariation,color="g") 
    plt.plot(p*100, W, 'ro')
    plt.grid("on")
    plt.legend(["Waiting time for CV="+"{:.2f}".format(cv_arrival),"Waiting time for CV=1"])
    plt.title("Waiting time based on utilization") 
    plt.xlabel("Utilization (%)")
    plt.ylabel("Waiting time (weeks)")
    plt.axvline(p*100, color='r',lw=0.5) # vertical
    plt.axhline(W, color='r',lw=0.5) # horizontal

In [182]:
def updatePlot(lambd,mu,cv_arrival): 
    cv_production=1
    p=lambd/mu # utilization in % 
    W=p/(1-p)*(np.power(cv_arrival,2)+np.power(cv_production,2))/2*1/mu
    p_chart=np.arange(0,1,0.01)
    W_chart=p_chart/(1-p_chart)*(np.power(cv_arrival,2)+np.power(cv_production,2))/2*1/mu
    W_chart_novariation=p_chart/(1-p_chart)*1/mu

    drawPlot(p,W,p_chart,W_chart,W_chart_novariation,cv_arrival) 
    plt.show()

    result=widgets.HTML(value="Expected waiting time: "+ "{:.2f}".format(W)+" weeks")
    display(result) 


In [183]:
def updateMaxLambdaSemicon(*args): 
    lambd_semicon.max=mu_semicon.value-0.1
    
def updateMaxLambdaOEM(*args): 
    lambd_oem.max=mu_oem.max-0.1
    
def updateMaxLambdaTier1(*args): 
    lambd_tier1.max=mu_tier1.max-0.1

def updateBW(*args): 
    cv_arrival_tier1.value=cv_arrival_oem.value*bullwhip_effect.value
    cv_arrival_semicon.value=cv_arrival_tier1.value*bullwhip_effect.value

def updateBWSemiconOnly(*args): 
    cv_arrival_semicon.value=cv_arrival_tier1.value*bullwhip_effect.value
    
    
mu_semicon.observe(updateMaxLambdaSemicon, 'value')
mu_tier1.observe(updateMaxLambdaTier1, 'value')
mu_oem.observe(updateMaxLambdaOEM, 'value')
bullwhip_effect.observe(updateBW,"value")
cv_arrival_tier1.observe(updateBWSemiconOnly,"value") 
cv_arrival_oem.observe(updateBW,"value") 
 

In [184]:
grid = GridspecLayout(3, 3,height="150px")
grid[0,:] = widgets.HTML(value="<b>Bullwhip-effect</b>") 
grid[1, :] = bullwhip_effect
grid[2,0]= widgets.HTML(value="<b>OEM</b>")
grid[2,1]= widgets.HTML(value="<b>Tier1-Supplier</b>")
grid[2,2]= widgets.HTML(value="<b>Semicon-Supplier</b>")
display(grid) 
grid2 = GridspecLayout(1, 3)
grid2[0, 0] =  widgets.interactive(updatePlot, lambd=lambd_oem,mu=mu_oem, cv_arrival=cv_arrival_oem,cv_production=1)
grid2[0, 1] =  widgets.interactive(updatePlot, lambd=lambd_tier1,mu=mu_tier1, cv_arrival=cv_arrival_tier1,cv_production=1)
grid2[0, 2] =  widgets.interactive(updatePlot, lambd=lambd_semicon,mu=mu_semicon, cv_arrival=cv_arrival_semicon,cv_production=1)
display(grid2) 



GridspecLayout(children=(HTML(value='<b>Bullwhip-effect</b>', layout=Layout(grid_area='widget001')), FloatSlid…

GridspecLayout(children=(interactive(children=(FloatSlider(value=1.4, description='Order arrival rate in k car…